In [69]:
#make files cross validation
import pandas as pd
import random
data = pd.read_csv('G:\\data analysis\\agaricus-lepiota.csv', sep=',')
t = list(range(len(data.values)))
header = list(data.columns)
for i in range(10):
    random.shuffle(t)
    file = open('G:\\data analysis\\train{}.csv'.format(i), 'w')
    k = t[0:50]
    file.write(','.join(header)+'\n')
    file.write( '\n'.join([','.join(list(val)) for val in data.values[k]]))
    k = t[50:100]
    file = open('G:\\data analysis\\test{}.csv'.format(i), 'w')
    file.write(','.join(header)+'\n')
    file.write( '\n'.join([','.join(list(val)) for val in data.values[k]]))
file.close()

In [9]:
def algorithm1(y_pred):
    for test_obj in X_test:
        pos = np.sum(test_obj == X_train_pos) / float(len(X_train_pos))
        neg = np.sum(test_obj == X_train_neg) / float(len(X_train_neg))
        if (pos > neg):
            y_pred.append(1)
        else:
            y_pred.append(0)



In [67]:
#алгоритм2
def algorithm2(y_pred):
    a =1.4
    sum1 = sum2 = 0
    for test in X_test:
        for pos in X_train_pos:
            tmp = test & pos
            sum1 += a**np.sum(test & pos)
            for neg in X_train_neg:
                if np.sum( tmp <= neg) == n:
                    sum1 -= a**np.sum(test & pos)
                    break
        for neg in X_train_neg:
            tmp = test & neg
            sum2+= a**np.sum(test & neg)
            for pos in X_train_pos:
                if np.sum( tmp <= pos) == n:
                    sum2 -= a**np.sum(test & neg)
                    break
        sum1 = sum1/(len(X_train_pos))
        sum2 = sum2/(len(X_train_neg))
        if sum1 > sum2:
            y_pred.append(1)
        else:
            y_pred.append(0)

In [10]:
#алгоритм3 байес
def algorithm3(y_pred):
    f_pos = [0 for i in range(n)]
    for pos in X_train_pos:
        for i in range(len(pos)):
            f_pos[i] += pos[i]
    f_pos = [tmp/len(X_train_pos) for tmp in f_pos]
    
    f_neg = [0 for i in range(n)]
    for neg in X_train_neg:
        for i in range(len(neg)):
            f_neg[i] += neg[i]
    f_neg = [tmp/len(X_train_neg) for tmp in f_neg]
    
    for test in X_test:
        sum1 = 1
        sum2 = 1
        for i in range(n):
            if test[i] == 1:
                sum1 *= f_pos[i]
                sum2 *= f_neg[i]
            else:
                sum1 *= (1 - f_pos[i])
                sum2 *= (1 - f_neg[i])
        sum1 = sum1*len(X_train_pos)
        sum2 = sum2*len(X_train_neg)
        if sum1 > sum2:
            y_pred.append(1)
        else:
            y_pred.append(0)     



In [11]:
#алгоритм4
def algorithm4(y_pred):
    for test in X_test:
        maxx = 0
        tmp = 0
        for neg in X_train_neg:
            if np.sum(test & neg) > maxx:
                maxx = np.sum(test & neg)
        for pos in X_train_pos:
            if np.sum(test & pos) > maxx:
                maxx = np.sum(test & pos)
                tmp = 1
        y_pred.append(tmp)

In [70]:
import pandas as pd
import numpy as np
import copy
features = [['b','c','x','f','k','s'],
            ['f','g','y','s'],
            ['n','b','c','g','r','p','u','e','w','y'], 
            ['t','f'], 
            ['a','l','c','y','f','m','n','p','s'],
            ['a','d','f','n'],
            ['c','w','d'], 
            ['b','n'], 
            ['k','n','b','h','g','r','o','p','u','e','w','y'],
            ['e','t'],
            ['b','c','u','e','z','r','?'], 
            ['f','y','k','s'], 
            ['f','y','k','s'],
            ['n','b','c','g','o','p','e','w','y'],
            ['n','b','c','g','o','p','e','w','y'],
            ['p','u'],
            ['n','o','w','y'],
            ['n','o','t'],
            ['c','e','f','l','n','p','s','z'],
            ['k','n','b','h','r','o','u','w','y'],
            ['a','c','n','s','v','y'],
            ['g','l','m','p','u','w','d']]
def dummy_encode_categorical_columns(data):
    result_data = copy.deepcopy(data)
    i = 0
    n = len(result_data.values)
    for column in data.columns.values:
        
        tmp = pd.get_dummies(list(result_data[column]) + features[i], prefix = column, prefix_sep = ': ')
        tmp = tmp.truncate(0, n-1)
        result_data = pd.concat([result_data, tmp], axis = 1)
        i += 1
        del result_data[column]
    return result_data
def parse_file(name):
    df = pd.read_csv(name, sep=',')
    #df[0] = df[0].replace(to_replace='e', value=1)
    for i in range(len(df['0'])):
        if df['0'][i] == 'e':
            df['0'][i] = 1
        else:
            df['0'][i] = 0
    #df[0] = df[0].replace(to_replace='p', value=0)
    y = np.array(df['0'])
    del df['0']
    bin_df = dummy_encode_categorical_columns(df)
    return np.array(bin_df).astype(int), y
TP = TN = FP = FN = TPR = TNR = FPR = NPV = FDR = acc = prec = rec = 0
for q in range(10):
    X_train, y_train = parse_file('G:\\data analysis\\train{}.csv'.format(q))
    X_test, y_test = parse_file('G:\\data analysis\\test{}.csv'.format(q))
    X_train_pos = X_train[y_train == 1]
    X_train_neg = X_train[y_train == 0]

    n = len(X_train[0])

    y_pred = []
    
    algorithm2(y_pred)

    y_pred = np.array(y_pred)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score

    y_test = y_test.astype(int)

    TP += np.sum(y_test * y_pred)
    TN += np.sum(y_test + y_pred == 0)
    FP += np.sum((y_test  == 0) * (y_pred == 1))
    FN += np.sum((y_test  == 1) * (y_pred == 0))
    TPR += float(TP) / np.sum(y_test == 1)
    TNR += float(TN) / np.sum(y_test == 0)
    FPR += float(FP) / (TP + FN)
    NPV += float(TN) / (TN + FN)
    FDR += float(FP) / (TP + FP)
    acc += accuracy_score(y_test, y_pred)
    prec += precision_score(y_test, y_pred)
    rec += recall_score(y_test, y_pred)

print("True Positive: {}\nTrue Negative: {}\nFalse Positive: {}\n\
False Negative: {}\nTrue Positive Rate: {}\nTrue Negative Rate: {}\n\
Negative Predictive Value: {}\nFalse Positive Rate: {}\nFalse Discovery Rate: {}\n\
Accuracy: {}\nAccuracyPrecision: {}\n\
Recall: {}".format(TP, TN, FP, FN, TPR/10, TNR/10, FPR/10, NPV/10, FDR/10, acc/10, prec/10, rec/10))

True Positive: 253
True Negative: 224
False Positive: 15
False Negative: 8
True Positive Rate: 5.484672364672365
True Negative Rate: 4.950305462653289
Negative Predictive Value: 0.07028366758195206
False Positive Rate: 0.9664852597743421
False Discovery Rate: 0.067294238581023
Accuracy: 0.9540000000000003
AccuracyPrecision: 0.9468703751239904
Recall: 0.9700783475783474
